In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/')
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['trade_data_defence_profit_loss_data.csv',
 'cal_data_defence_profit_loss_data.csv',
 'ideal_data_defence_profit_loss_data.csv']

In [ ]:
trade_data = pd.read_csv("trade_data_defence_profit_loss_data.csv")
cal_data= pd.read_csv("cal_data_defence_profit_loss_data.csv")
ideal_data = pd.read_csv("ideal_data_defence_profit_loss_data.csv")

In [ ]:
trade_data.drop('Unnamed: 0',axis=1,inplace=True)
ideal_data.drop('Unnamed: 0',axis=1,inplace=True)
cal_data.drop('Unnamed: 0',axis=1,inplace=True)
trade_data["Time Stamp"] = pd.to_datetime(trade_data["Time Stamp"])
ideal_data["Time Stamp"] = pd.to_datetime(ideal_data["Time Stamp"])
cal_data["Time Stamp"] = pd.to_datetime(cal_data["Time Stamp"])

In [ ]:
cal_data.head()

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry_C,Defence_C,Stop_C,Target_C,Profit_C,Loss_C,defence_target_C,D_Profit_C,D_Loss_C
0,2019-06-23 18:20:00,3235.50,3235.50,3234.75,3235.50,0.0,0.0,0.0,0.0,0.00,3230.0,0.00,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
1,2019-06-23 18:25:00,3235.50,3235.75,3234.50,3234.50,0.0,0.0,0.0,0.0,3235.75,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
2,2019-06-23 18:30:00,3234.75,3235.75,3234.50,3235.75,0.0,0.0,0.0,0.0,3235.75,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
3,2019-06-23 18:35:00,3235.75,3236.50,3235.25,3236.50,0.0,0.0,0.0,0.0,0.00,3230.0,3234.25,0.00,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
4,2019-06-23 18:40:00,3236.50,3236.50,3235.75,3236.00,0.0,0.0,0.0,0.0,0.00,3230.0,3234.25,3235.75,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0


In [ ]:
cal_data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry_C', 'Defence_C',
       'Stop_C', 'Target_C', 'Profit_C', 'Loss_C', 'defence_target_C',
       'D_Profit_C', 'D_Loss_C'],
      dtype='object')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Preprocess your data
columns_to_predict = ['Defence_C', 'Stop_C', 'Target_C','defence_target_C']
feature_columns = ['Open', 'High', 'Low', 'Close', 'SellersLevel4',
                   'SellersLevel3', 'SellersLevel2', 'SellersLevel1',
                   'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1',
                   'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4',
                   'BuyersInControlPrice', 'SellersInControlPrice',
                   'Entry_C', 'Defence_C', 'Stop_C', 'Target_C']

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(cal_data[feature_columns])

# Create sequences
def create_sequences(data, seq_length=10):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length, -4:]  # Last three columns to predict
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequence_length = 10
X, y = create_sequences(scaled_data, sequence_length)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=3))  # 3 outputs for 'Defence_T', 'Stop_T', 'Target_T'

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Make predictions
predictions = model.predict(X_test)
# Inverse transform the predictions to get the original scale
predictions = scaler.inverse_transform(np.concatenate((np.zeros((predictions.shape[0], scaled_data.shape[1]-3)), predictions), axis=1))[:, -3:]

print("Predictions:", predictions)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


ValueError: Dimensions must be equal, but are 4 and 3 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, sequential_1/dense_1/Add)' with input shapes: [?,4], [?,3].

In [ ]:
cal_data.drop(['BuyersInControlTime','SellersInControlTime','Profit_C','Loss_C','D_Profit_C','D_Loss_C'],axis=1,inplace=True)
cal_data.columns

cal_data.shape

cal_data.set_index('Time Stamp', inplace=True)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Split the data
train_data = cal_data[cal_data.index < '2023-06-23 00:05:00']
test_data = cal_data[cal_data.index >= '2023-06-24 00:05:00']


# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(train_data)

# Preparing the sequences
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        target = data[i+sequence_length, -4:]  # Assuming last 4 columns are the targets
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)



